In [ ]:
import h5py
import numpy as np
from astropy.table import Table, vstack
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table
import numpy as np
import math
import warnings
import matplotlib.pyplot as plt
import glob


# Methods

In [ ]:
def reweight_gaug(data, zenith, alpha  = 0.7, is_data=False):
    if is_data:
        weights = np.ones(len(data))/len(data)
    else:
        zenith = np.deg2rad(zenith)
        weights = np.ones(len(data))
        weights = data['mc_energy']**(-alpha) * (1/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(np.deg2rad(zenith))/115) + 0.054/(1 + 1.1 * data['mc_energy'] * 1000 * np.cos(np.deg2rad(zenith))/850))
    return weights / np.sum(weights)

def check_h5_structure(file_path):
    with h5py.File(file_path, "r") as h5_file:
        
        # List all groups in the file
        print("Groups in the file:")
        for group in h5_file:
            print(group)
        
        # You can also recursively list all datasets
        def print_structure(name, obj):
            print(f"{name}: {obj}")
        
        print("\nFile structure:")
        h5_file.visititems(print_structure)

def read_and_process_fits(files):
    dat = None
    for muon_file in files:
        dat2 = Table.read(muon_file, format='fits')
        
        # Convert columns to boolean if they exist
        for col in ['good_ring', 'is_valid', 'parameters_at_limit']:
            if col in dat2.colnames:
                dat2[col] = dat2[col].astype(bool)
        
        # Stack the tables
        dat = vstack([dat, dat2]) if dat is not None else dat2
    
    return dat

def plot_columns(df_lstchain, df_ctapipe, columns_lstchain, columns_ctapipe, rows, cols, bin_power = 0.4, apply_weight=True, plot_option=True):
    fig, axes = plt.subplots(rows, cols, figsize=(12, 18))
    axes = axes.flatten()
    
    for i, col in enumerate(columns_lstchain):
        lstchain_values = df_lstchain[col][~np.isnan(df_lstchain[col])]
        ctapipe_values = df_ctapipe[columns_ctapipe[i]][~np.isnan(df_ctapipe[columns_ctapipe[i]])]
    

        min_value = min(min(ctapipe_values), min(lstchain_values))
        max_value = max(max(ctapipe_values), max(lstchain_values))

        num_bins = int(max(len(df_lstchain)**bin_power, len(df_ctapipe))**bin_power)
        #num_bins = max(10, min(num_bins, 50))

        
        bins = np.linspace(min_value, max_value, num_bins + 1)

        if apply_weight:
            lst_weights = reweight_gaug(df_lstchain, 10, 0.7, False)
            ctapipe_weights = reweight_gaug(df_ctapipe, 10, 0.7, False)
            axes[i].hist(lstchain_values, weights=lst_weights, bins=bins, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
            axes[i].hist(ctapipe_values, weights=ctapipe_weights, bins=bins, alpha=0.85, color='orange', density=True, label=f'Muon Simulation 0.0046deg', histtype='step', lw=2)
        else:
            axes[i].hist(lstchain_values, bins=bins, alpha=0.6, color='k', density=True, label='lstchain', histtype='step', lw=2)
            axes[i].hist(ctapipe_values, bins=bins, alpha=0.85, color='orange', density=True, label=f'ctapipe', histtype='step', lw=2)

        axes[i].set_title(f'Histogram of {col}')
        axes[i].set_xlabel(col)
        axes[i].set_ylabel('Normalized Frequency')
        axes[i].legend(loc='upper right')

    plt.tight_layout()
    if plot_option:
        plt.savefig(f'/Users/vdk/muons2024/basic_distributions/PROTONS_noringcenter_{name}_radiuscut={min_radius_cut}-{max_radius_cut}_completeness={ring_completeness_cut}_centerdistance={ring_center_distance_cut}_impactdistance={impact_distance_cut}_widthquantile={QUANTILE_CUT}.pdf', dpi=200)
    plt.show()

comparison_columns_lstchain = [
    'ring_size', 
    'ring_center_x', 
    'ring_center_y', 
    'ring_radius',  
    'ring_width', 
    'impact_parameter', 
    'muon_efficiency',
    'radial_stdev',
    ]
comparison_columns_ctapipe = [
    'muonparameters_ring_size', 
    'muonring_center_fov_lon', 
    'muonring_center_fov_lat', 
    'muonring_radius',  
    'muonefficiency_width', 
    'muonefficiency_impact',
    'muonefficiency_optical_efficiency',
    'muonparameters_standard_dev'
]

# ctapipe data

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/ctapipe/data/16903'
dl1_files = glob.glob(folder_path + '/*dl1.h5')

tables = []
for file_path in dl1_files:
    with h5py.File(file_path, "r") as h5_file:
        dataset = h5_file["dl1/event/telescope/muon/tel_001"]
        dataset_dtype = dataset.dtype
        data = dataset[:]
    
    data_array = np.array(data, dtype=dataset_dtype)
    muon_table = Table(data_array)
    tables.append(muon_table)

muon_table_ctapipe_data_16903 = vstack(tables)
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_is_valid'] == 1]
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_optical_efficiency'] > 0]
muon_table_ctapipe_data_16903

In [ ]:
muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonparameters_ring_size'] > 0]

In [ ]:
#muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonefficiency_is_valid'] == 1]
plt.figure(figsize=(12, 8))
#z,x,c = plt.hist(muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonparameters_ring_size'] > 0]['muonparameters_ring_size'], bins = 500, alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
z,x,c = plt.hist(muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonparameters_ring_size'] > 0]['muonparameters_ring_size'], bins = np.linspace(0,100000,100), alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
plt.xlabel('Ring Size')
plt.ylabel('Normalized Frequency')

In [ ]:
plt.figure(figsize=(12, 8))
z,x,c = plt.hist(muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonparameters_ring_size'] > 0]['muonring_radius'], bins = np.linspace(0,2,50), alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
plt.xlabel('Ring Radius')
plt.ylabel('Normalized Frequency')

In [ ]:
plt.figure(figsize=(12, 8))
z,x,c = plt.hist(muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonparameters_ring_size'] > 0]['muonparameters_mean_squared_error'], bins = np.linspace(0,0.5,100), alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
plt.xlabel('MSE of ring geometry fit')
plt.ylabel('Normalized Frequency')

In [ ]:
plt.figure(figsize=(12, 8))
z,x,c = plt.hist(muon_table_ctapipe_data_16903[muon_table_ctapipe_data_16903['muonparameters_ring_size'] > 0]['muonparameters_size_outside'], bins = np.linspace(0,1000,100), alpha=0.6, color='k', density=True, label='Data', histtype='step', lw=2)
plt.xlabel('Size Outside')
plt.ylabel('Normalized Frequency')

In [ ]:
with h5py.File(dl1_files[0], "r") as h5_file:
    dataset = h5_file["dl1/event/telescope/muon/tel_001"]
    muon_dataset_dtype = dataset.dtype
    muon_data = dataset[:]
    dataset = h5_file["/dl1/event/telescope/images/tel_001"]
    image_dataset_dtype = dataset.dtype
    image_data = dataset[:]

muon_table = Table(np.array(muon_data, dtype=muon_dataset_dtype))
image_table = Table(np.array(image_data, dtype=image_dataset_dtype))

In [ ]:
image_table[image_table['event_id'] == 1196156]['image']
image_table

In [ ]:
muon_mask = ~np.isnan(muon_table['muonparameters_ring_size'])
muon_table[muon_mask]['event_id']

In [ ]:
image_mask = np.isin(image_table['event_id'], muon_table[muon_mask]['event_id'])
image_table[image_mask]

In [ ]:
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay

source = EventSource('/Users/vdk/muons2024/simtel_files/2024year_tuned_nooulier_reflectivity_additional/run101_muon.simtel.gz')

camgeom = source.subarray.tel[1].camera.geometry
for i in range(1098):
    if np.sum(image_table[image_mask]['image'][i]) < 100000 and muon_table[muon_mask]['muonparameters_num_pixels_in_ring'][i] > 95 and muon_table[muon_mask]['muonparameters_num_pixels_in_ring'][i] < 180:
        disp = CameraDisplay(camgeom, image=image_table[image_mask]['image'][i])
        disp.cmap = plt.cm.RdBu_r
        disp.add_colorbar()
        disp.set_limits_percent(95)
        plt.title(f"{np.sum(image_table[image_mask]['image'][i])}, {muon_table[muon_mask]['muonparameters_num_pixels_in_ring'][i]}")

        plt.show()

In [ ]:
muon_table[muon_table['muonefficiency_is_valid'] == 1]

# ctapipe sim std

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/ctapipe/sim/ctapipe_00046deg'
dl1_files = glob.glob(folder_path + '/dl1*.h5')

tables = []
for file_path in dl1_files:
    with h5py.File(file_path, "r") as h5_file:
        dataset = h5_file["dl1/event/telescope/muon/tel_001"]
        dataset_dtype = dataset.dtype
        data = dataset[:]
    
    data_array = np.array(data, dtype=dataset_dtype)
    muon_table = Table(data_array)
    tables.append(muon_table)

muon_table_ctapipe_00046 = vstack(tables)
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_is_valid'] == 1]
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_optical_efficiency'] > 0]
muon_table_ctapipe_00046

# ctapipe sim 0.015

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/ctapipe/sim/ctapipe_0015deg'
dl1_files = glob.glob(folder_path + '/dl1*.h5')

tables = []
for file_path in dl1_files:
    with h5py.File(file_path, "r") as h5_file:
        dataset = h5_file["dl1/event/telescope/muon/tel_001"]
        dataset_dtype = dataset.dtype
        data = dataset[:]
    
    data_array = np.array(data, dtype=dataset_dtype)
    muon_table = Table(data_array)
    tables.append(muon_table)

muon_table_ctapipe_0015 = vstack(tables)
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_is_valid'] == 1]
#muon_table_ctapipe[muon_table_ctapipe['muonefficiency_optical_efficiency'] > 0]
muon_table_ctapipe_0015

# lstchain data effective

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/lstchain/data/16903/effective_focal_length/tables/*'
muon_table_lstchain_data_16903_eff = read_and_process_fits(glob.glob(folder_path))
muon_table_lstchain_data_16903_eff

# lstchain data equivalent

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/lstchain/data/16903/equivalent_focal_length/*'
muon_table_lstchain_data_16903_eq = read_and_process_fits(glob.glob(folder_path))
muon_table_lstchain_data_16903_eq

# lstchain sim std effective

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/lstchain/sim/effective_focal_length/00046deg/*'
muon_table_lstchain_data_00046_eff = read_and_process_fits(glob.glob(folder_path))
muon_table_lstchain_data_00046_eff

# lstchain sim std equivalent

# lstchain sim 0.015 effective

In [ ]:
folder_path = '/Users/vdk/muons2024/focal_difference/lstchain/sim/effective_focal_length/0015deg/*'
muon_table_lstchain_data_0015_eff = read_and_process_fits(glob.glob(folder_path))
muon_table_lstchain_data_0015_eff

# lstchain sim 0.015 equivalent